In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import torch
from hydra.utils import instantiate
from hydra import initialize, compose
import hydra

import wandb

from data.dataManager import DataManager
from model.modelCreator import ModelCreator
from omegaconf import OmegaConf
from scripts.run import setup_model, load_model_instance

from utils.plots import vae_plots
from utils.rbm_plots import plot_rbm_histogram, plot_rbm_params, plot_forward_output_v2

from scripts.run import set_device

[20:04:31.451] INFO   CaloQuVAE                                         Loading configuration.


In [2]:
hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(version_base=None, config_path="config")
config=compose(config_name="config.yaml")
wandb.init(tags = [config.data.dataset_name], project=config.wandb.project, entity=config.wandb.entity, config=OmegaConf.to_container(config, resolve=True), mode='disabled')

In [3]:
new_model = True
if new_model:
    self = setup_model(config)
    # self.model = self.model.double()  # sets all model parameters to float64
else:
    self = load_model_instance(config)
    # self.model = self.model.double()


[20:04:32.650] INFO   data.dataManager                                  Loading other dataset: CaloChallenge2
[20:04:32.654] INFO   data.dataManager                                  Keys: ['incident_energies', 'showers']


[20:04:37.608] INFO   data.dataManager                                  dict_keys(['incident_energies', 'showers'])
[20:04:37.610] INFO   data.dataManager                                  <torch.utils.data.dataloader.DataLoader object at 0x7f5ca97d1e50>: 79999 events, 157 batches
[20:04:37.610] INFO   data.dataManager                                  <torch.utils.data.dataloader.DataLoader object at 0x7f5a1f0aae10>: 10001 events, 10 batches
[20:04:37.610] INFO   data.dataManager                                  <torch.utils.data.dataloader.DataLoader object at 0x7f5a1fbc2b70>: 9999 events, 10 batches
[20:04:37.611] INFO   model.modelCreator                                ::Creating Model
[20:04:38.351] INFO   dwave.cloud.client.base                           Fetching definitions of all available solvers
[20:04:38.395] INFO   dwave.cloud.client.base                           Received solver data for 7 solver(s).
[20:04:38.638] INFO   dwave.cloud.client.base                           Add

RBM_Hiddentorch initialized
cuda:0
encoder._networks.0.seq1.0.conv.weight True
encoder._networks.0.seq1.0.conv.bias True
encoder._networks.0.seq1.1.weight True
encoder._networks.0.seq1.1.bias True
encoder._networks.0.seq1.2.weight True
encoder._networks.0.seq1.3.conv.weight True
encoder._networks.0.seq1.3.conv.bias True
encoder._networks.0.seq1.4.weight True
encoder._networks.0.seq1.4.bias True
encoder._networks.0.seq1.5.weight True
encoder._networks.0.seq1.6.conv.weight True
encoder._networks.0.seq1.6.conv.bias True
encoder._networks.0.seq1.7.weight True
encoder._networks.0.seq1.7.bias True
encoder._networks.0.seq1.8.weight True
encoder._networks.0.seq2.0.conv.weight True
encoder._networks.0.seq2.0.conv.bias True
encoder._networks.0.seq2.1.weight True
encoder._networks.0.seq2.1.bias True
encoder._networks.0.seq2.2.weight True
encoder._networks.0.seq2.3.conv.weight True
encoder._networks.0.seq2.3.conv.bias True
encoder._networks.0.seq2.4.weight True
encoder._networks.1.seq1.0.conv.weig

In [4]:
self.evaluate_vae(self.data_mgr.val_loader,0)

[20:05:17.968] INFO   engine.engine                                     Epoch: 0 - Average Val Loss: 62132.4348


{'val_ae_loss': 58179.002734375,
 'val_kl_loss': 177.08904418945312,
 'val_hit_loss': 3776.3433349609377,
 'val_entropy': -418.1646270751953,
 'val_pos_energy': -255.81539154052734,
 'val_neg_energy': 851.0690673828125,
 'val_loss': 62132.434765625}